Name: Erik Alexander Sandvik <br />
Course: STK-IN4300 <br />
Assignment number: 2 <br />

In [108]:
%matplotlib inline

# Import required packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Problem 1. Regression

### 1.

In [109]:
# Read data from file

df = pd.read_csv("qsar_aquatic_toxicity.csv", sep=";")


# Label the columns

names = ['TPSA', 'SAacc', 'H050', 'MLOGP', 'RDCHI', 'GATS1p', 'nN', 'C040', 'LC50']
df.columns = names


# Split dataframe by features and response variable

X = df.iloc[:, :-1]
y = df.iloc[:, -1]


# Split the data into a test and training set

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)


# Ordinary Least Squares

from sklearn import linear_model
reg = linear_model.LinearRegression()

reg.fit(X_train, y_train)


# Calculate the predicted response using the training set and test set as inputs

y_predict_train = reg.predict(X_train)
y_predict_test = reg.predict(X_test)


# Calculate the training and test error

from sklearn.metrics import mean_squared_error as MSE

print('Training error: {:.2f}'.format(MSE(y_train, y_predict_train)))
print('Test error: {:.2f}'.format(MSE(y_test, y_predict_test)))

Training error: 1.20
Test error: 1.92


In [110]:
# Calculate the significance level of each coefficient in the linear model
# scikit-learn doesn't have this implemented

import statsmodels.api as sm

mod = sm.OLS(y_train, X_train)
fii = mod.fit()
p_values = fii.summary2().tables[1]['P>|t|']

print(p_values)

TPSA      7.995407e-20
SAacc     4.212188e-19
H050      1.530054e-03
MLOGP     2.360072e-15
RDCHI     1.685756e-07
GATS1p    7.462060e-04
nN        1.525479e-02
C040      6.835107e-01
Name: P>|t|, dtype: float64


In [111]:
# Same thing as above, but with dichotomization of the count variables

count_variables = ['H050', 'nN', 'C040']

for cnt_var in count_variables:
    X.loc[X[cnt_var] > 0, cnt_var] = 1
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

reg.fit(X_train, y_train)

y_predict_train = reg.predict(X_train)
y_predict_test = reg.predict(X_test)

print('Training error: {:.2f}'.format(MSE(y_train, y_predict_train)))
print('Test error: {:.2f}'.format(MSE(y_test, y_predict_test)))

mod = sm.OLS(y_train, X_train)
fii = mod.fit()
p_values = fii.summary2().tables[1]['P>|t|']

print('\n')
print(p_values)

Training error: 1.22
Test error: 2.02


TPSA      1.601354e-15
SAacc     3.631432e-15
H050      1.474690e-01
MLOGP     6.194481e-15
RDCHI     5.536240e-07
GATS1p    3.161653e-03
nN        2.888570e-01
C040      2.689950e-01
Name: P>|t|, dtype: float64


With dichotomization the test error increases and the p-values of the coefficients increases by orders of magnitude, with the exception of the variable 'C040' where the p-value of the corresponding coefficient is reduced by a factor of 2.

### 2.